# XEROPHYTA ELEGANS

In [1]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [48]:
tr = pd.read_csv("/home/t44p/PW_rawdata/Transciptome_GenomeAnnotation/Normalised_counts_table.csv", sep=",")
lcms = pd.read_csv("/home/t44p/PW_rawdata/LC-MS/20230908_Final peaks_imp+log2+Combat_ME.csv", sep=",")
gcms = pd.read_csv("/home/t44p/PW_rawdata/GC-MS/gc-ms.csv", sep=",")

## Data Prep


In [49]:
print(tr.columns)
#print(lcms.columns)
#print(gcms.columns)

for col in gcms.columns:
    print(col)

Index(['Unnamed: 0', 'Xe_De_R2_T00', 'Xe_De_R2_T03', 'Xe_De_R2_T06',
       'Xe_De_R2_T09', 'Xe_De_R2_T12', 'Xe_De_R2_T15', 'Xe_De_R2_T24',
       'Xe_De_R3_T00', 'Xe_De_R3_T03', 'Xe_De_R3_T06', 'Xe_De_R3_T09',
       'Xe_De_R3_T12', 'Xe_De_R3_T15', 'Xe_De_R3_T24', 'Xe_De_R4_T00',
       'Xe_De_R4_T03', 'Xe_De_R4_T06', 'Xe_De_R4_T09', 'Xe_De_R4_T12',
       'Xe_De_R4_T15', 'Xe_De_R4_T24', 'Xe_Re_R2_T00', 'Xe_Re_R2_T01',
       'Xe_Re_R2_T02', 'Xe_Re_R2_T04', 'Xe_Re_R2_T08', 'Xe_Re_R2_T12',
       'Xe_Re_R2_T48', 'Xe_Re_R6_T00', 'Xe_Re_R4_T01', 'Xe_Re_R3_T02',
       'Xe_Re_R3_T04', 'Xe_Re_R3_T08', 'Xe_Re_R3_T12', 'Xe_Re_R3_T48',
       'Xe_Re_R7_T00', 'Xe_Re_R5_T01', 'Xe_Re_R5_T02', 'Xe_Re_R5_T04',
       'Xe_Re_R5_T08', 'Xe_Re_R5_T12', 'Xe_Re_R5_T48'],
      dtype='object')
Unnamed: 0
Xe_De_R2_T00
Xe_De_R2_T03
Xe_De_R2_T06
Xe_De_R2_T09
Xe_De_R2_T12
Xe_De_R2_T24
Xe_De_R3_T00
Xe_De_R3_T03
Xe_De_R3_T06
Xe_De_R3_T09
Xe_De_R3_T12
Xe_De_R3_T15
Xe_De_R3_T24
Xe_De_R4_T00
Xe_De_R4_T03
Xe_De_R4

## Reconcile Transcriptome Timepoints & LC/GC-MS

> gcms has duplicate column names 
- due to naming inconsistence: Xe_Re_6_T00 vs. Xe_Re_R6_T00, Xe_Re_7_T00 vs. Xe_Re_R7_T00
- Xe_Re_R6_T00 count: 2
- Xe_Re_R7_T00 count: 2

In [50]:
# rename 1st column "unamed" -> "ref"
tr_cols = np.array(tr.columns)
tr_cols[0] = 'ref'
tr.columns = tr_cols

gcms_cols = np.array(gcms.columns)
gcms_cols[0] = 'metabolite'
gcms.columns = gcms_cols


In [51]:
# find mutual columns
tr_cols = set(tr.columns[1:])
gmcs_cols = set(gcms.columns[1:])

# find cols from transcriptome that are not in GC-MS
tr_notin_gcms_cols = list(tr_cols.difference(gcms_cols))

# find mutual column names
mutual_cols = tr_cols.intersection(gmcs_cols) 
mutual_cols = sorted(list(mutual_cols))
tr_mut = pd.DataFrame(tr['ref'])
tr_mut = tr_mut.join(tr[mutual_cols])

gcms_mut = pd.DataFrame(gcms['metabolite'])
gcms_mut = gcms_mut.join(gcms[mutual_cols])

#gcms_mut.to_csv('/home/t44p/PW_rawdata/tr_gc_mutual/gcms_mut.csv', index=False)
#tr_mut.to_csv('/home/t44p/PW_rawdata/tr_gc_mutual/tr_mut.csv', index=False)



In [52]:

#lcms_cols = np.array(lcms.columns)
#lcms_cols[0] = 'metabolite'
#lcms.columns = lcms_cols

# find mutual columns
#tr_cols = set(tr.columns[1:])
#lmcs_cols = set(lcms.columns[1:])

# find cols from transcriptome that are not in GC-MS
#tr_notin_lcms_cols = list(tr_cols.difference(lcms_cols))

# find mutual column names
#lcms_mutual_cols = tr_cols.intersection(lmcs_cols) 
#lcms_mutual_cols = sorted(list(lcms_mutual_cols))
#tr_mut = pd.DataFrame(tr['ref'])
#tr_mut = tr_mut.join(tr[lcms_mutual_cols])

# I WILL USE GCMS & TRANSCRIPTOME INTERSECTION AS REFERENCE FOR LCMS
# LCMS DOES SHARE ON MORE COLUMN WITH TRANSCRIPTOME DATA THAN GCMS DOES
# Xe_De_R2_T15

lcms_mut = pd.DataFrame(lcms['Metabolites'])
lcms_mut.columns = ['metabolite'] + lcms_mut.columns[1:].tolist()
lcms_mut = lcms_mut.join(lcms[mutual_cols])


# FLOATS ARE RECOGNIZE AS STRINGS, CONVERT TO STRING TO FLOAT
for col in lcms_mut.columns[1:]:
    lcms_mut[col] = pd.to_numeric(lcms_mut[col], errors='coerce')
lcms_mut.to_csv('/home/t44p/PW_rawdata/tr_gc_mutual/lcms_mut.csv', index=False)



In [46]:
set(lcms_mut.columns).intersection(set(gcms_mut.columns))

{'Xe_De_R2_T00',
 'Xe_De_R2_T03',
 'Xe_De_R2_T06',
 'Xe_De_R2_T09',
 'Xe_De_R2_T12',
 'Xe_De_R2_T24',
 'Xe_De_R3_T00',
 'Xe_De_R3_T03',
 'Xe_De_R3_T06',
 'Xe_De_R3_T09',
 'Xe_De_R3_T12',
 'Xe_De_R3_T15',
 'Xe_De_R3_T24',
 'Xe_De_R4_T00',
 'Xe_De_R4_T03',
 'Xe_De_R4_T06',
 'Xe_De_R4_T09',
 'Xe_De_R4_T12',
 'Xe_De_R4_T15',
 'Xe_De_R4_T24',
 'Xe_Re_R2_T00',
 'Xe_Re_R2_T01',
 'Xe_Re_R2_T02',
 'Xe_Re_R2_T04',
 'Xe_Re_R2_T08',
 'Xe_Re_R2_T12',
 'Xe_Re_R2_T48',
 'Xe_Re_R3_T02',
 'Xe_Re_R3_T04',
 'Xe_Re_R3_T08',
 'Xe_Re_R3_T12',
 'Xe_Re_R3_T48',
 'Xe_Re_R5_T01',
 'Xe_Re_R5_T02',
 'Xe_Re_R5_T04',
 'Xe_Re_R5_T08',
 'Xe_Re_R5_T12',
 'Xe_Re_R5_T48',
 'Xe_Re_R6_T00',
 'Xe_Re_R7_T00',
 'metabolite'}

## GC-MS data:

### Account for inconsistent column names 
### Xe_De_6_T12 to Xe_De_R6_T12"
> will produce duplicates for following
- Xe_Re_6_T00 vs. Xe_Re_R6_T00, Xe_Re_7_T00 vs. Xe_Re_R7_T00

In [98]:
import re

def replace_third_column(s):
    # regex matches an underscore, a single digit, and another underscore
    pattern = r'(_\d_)'
    # This function replaces the matched pattern with _R and the matched digit
    replacer = lambda x: '_R' + x.group(1)[1] + "_"
    # Replace the third occurrence of the pattern
    return re.sub(pattern, replacer, s, count=3)

# Example usage
#s = "Xe_De_6_T12"
#modified_s = replace_third_column(s)
#print(modified_s)
gcms_col_array = np.array(gcms.columns)
gcms_col_array = np.array([replace_third_column(col) for col in gcms_col_array])
uneq=[a != b for a, b in zip(gcms_col_array, gcms.columns)] # check where gcms.columns and gcms_col_arry differ
# compare the orignal columns names with the processed columns
print("unprocessed >>>/n", np.array(gcms.columns[uneq]), "length >>> ", len(gcms.columns))
print("procesed >>>/n", gcms_col_array[uneq], "length >>> ", len(gcms_col_array))

unprocessed >>>/n ['Xe_De_5_T00' 'Xe_De_5_T03' 'Xe_De_5_T06' 'Xe_De_5_T09' 'Xe_De_5_T12'
 'Xe_De_5_T15' 'Xe_De_5_T24' 'Xe_De_6_T12' 'Xe_De_6_T15' 'Xe_De_8_T00'
 'Xe_De_8_T03' 'Xe_De_8_T06' 'Xe_De_8_T15' 'Xe_De_8_T24' 'Xe_De_9_T00'
 'Xe_De_9_T03' 'Xe_De_9_T06' 'Xe_De_9_T09' 'Xe_De_9_T12' 'Xe_De_9_T15'
 'Xe_De_9_T24' 'Xe_Re_4_T00' 'Xe_Re_4_T02' 'Xe_Re_4_T04' 'Xe_Re_4_T08'
 'Xe_Re_4_T12' 'Xe_Re_4_T48' 'Xe_Re_5_T00' 'Xe_Re_6_T00' 'Xe_Re_6_T01'
 'Xe_Re_6_T02' 'Xe_Re_6_T04' 'Xe_Re_6_T08' 'Xe_Re_6_T12' 'Xe_Re_6_T48'
 'Xe_Re_7_T00' 'Xe_Re_7_T01' 'Xe_Re_7_T02' 'Xe_Re_7_T04' 'Xe_Re_7_T08'
 'Xe_Re_7_T12' 'Xe_Re_7_T48' 'Xe_Re_8_T01'] length >>>  85
procesed >>>/n ['Xe_De_R5_T00' 'Xe_De_R5_T03' 'Xe_De_R5_T06' 'Xe_De_R5_T09'
 'Xe_De_R5_T12' 'Xe_De_R5_T15' 'Xe_De_R5_T24' 'Xe_De_R6_T12'
 'Xe_De_R6_T15' 'Xe_De_R8_T00' 'Xe_De_R8_T03' 'Xe_De_R8_T06'
 'Xe_De_R8_T15' 'Xe_De_R8_T24' 'Xe_De_R9_T00' 'Xe_De_R9_T03'
 'Xe_De_R9_T06' 'Xe_De_R9_T09' 'Xe_De_R9_T12' 'Xe_De_R9_T15'
 'Xe_De_R9_T24' 'Xe_Re_R4_T00' 'Xe_

In [99]:
gcms_col_array[0] = "metabolite"
gcms.columns = gcms_col_array
#gcms.to_csv('/home/t44p/PW_rawdata/GC-MS/gc-ms_prep.csv', index=False)

### sort columns by replicates

In [100]:
def extract_r_number(column_name):
    """ Extracts the 'R[0-9]' part from the column name. """
    match = re.search(r'R\d+', column_name)
    return match.group(0) if match else '' 
# group() allows you to extract the specific portions of the string that matched the pattern.

gcms_col_array_sorted = sorted(gcms_col_array, key=extract_r_number)
for col in gcms_col_array_sorted:
    print(col)

metabolite
Xe_De_R2_T00
Xe_De_R2_T03
Xe_De_R2_T06
Xe_De_R2_T09
Xe_De_R2_T12
Xe_De_R2_T24
Xe_Re_R2_T00
Xe_Re_R2_T01
Xe_Re_R2_T02
Xe_Re_R2_T04
Xe_Re_R2_T08
Xe_Re_R2_T12
Xe_Re_R2_T48
Xe_De_R3_T00
Xe_De_R3_T03
Xe_De_R3_T06
Xe_De_R3_T09
Xe_De_R3_T12
Xe_De_R3_T15
Xe_De_R3_T24
Xe_Re_R3_T01
Xe_Re_R3_T02
Xe_Re_R3_T04
Xe_Re_R3_T08
Xe_Re_R3_T12
Xe_Re_R3_T48
Xe_De_R4_T00
Xe_De_R4_T03
Xe_De_R4_T06
Xe_De_R4_T09
Xe_De_R4_T12
Xe_De_R4_T15
Xe_De_R4_T24
Xe_Re_R4_T00
Xe_Re_R4_T02
Xe_Re_R4_T04
Xe_Re_R4_T08
Xe_Re_R4_T12
Xe_Re_R4_T48
Xe_Re_R5_T01
Xe_Re_R5_T02
Xe_Re_R5_T04
Xe_Re_R5_T08
Xe_Re_R5_T12
Xe_Re_R5_T48
Xe_De_R5_T00
Xe_De_R5_T03
Xe_De_R5_T06
Xe_De_R5_T09
Xe_De_R5_T12
Xe_De_R5_T15
Xe_De_R5_T24
Xe_Re_R5_T00
Xe_Re_R6_T00
Xe_De_R6_T12
Xe_De_R6_T15
Xe_Re_R6_T00
Xe_Re_R6_T01
Xe_Re_R6_T02
Xe_Re_R6_T04
Xe_Re_R6_T08
Xe_Re_R6_T12
Xe_Re_R6_T48
Xe_Re_R7_T00
Xe_Re_R7_T00
Xe_Re_R7_T01
Xe_Re_R7_T02
Xe_Re_R7_T04
Xe_Re_R7_T08
Xe_Re_R7_T12
Xe_Re_R7_T48
Xe_De_R8_T00
Xe_De_R8_T03
Xe_De_R8_T06
Xe_De_R8_T15
Xe_De_R8_T24
X

## LC-MS data:
### Account for inconsistentce column names 
### Xe_De_6_T12 to Xe_De_R6_T12"



In [101]:
lcms.columns

Index(['Metabolites', 'Xe_De_R2_T00', 'Xe_De_R3_T00', 'Xe_De_R4_T00',
       'Xe_De_5_T00', 'Xe_De_8_T00', 'Xe_De_9_T00', 'Xe_De_R2_T03',
       'Xe_De_R3_T03', 'Xe_De_R4_T03', 'Xe_De_5_T03', 'Xe_De_8_T03',
       'Xe_De_9_T03', 'Xe_De_R2_T06', 'Xe_De_R3_T06', 'Xe_De_R4_T06',
       'Xe_De_5_T06', 'Xe_De_8_T06', 'Xe_De_9_T06', 'Xe_De_R2_T09',
       'Xe_De_R3_T09', 'Xe_De_R4_T09', 'Xe_De_5_T09', 'Xe_De_8_T09',
       'Xe_De_9_T09', 'Xe_De_R2_T12', 'Xe_De_R3_T12', 'Xe_De_R4_T12',
       'Xe_De_5_T12', 'Xe_De_6_T12', 'Xe_De_9_T12', 'Xe_De_R2_T15',
       'Xe_De_R3_T15', 'Xe_De_R4_T15', 'Xe_De_5_T15', 'Xe_De_6_T15',
       'Xe_De_8_T15', 'Xe_De_9_T15', 'Xe_De_R2_T24', 'Xe_De_R3_T24',
       'Xe_De_R4_T24', 'Xe_De_5_T24', 'Xe_De_8_T24', 'Xe_De_9_T24',
       'Xe_Re_R2_T00', 'Xe_Re_4_T00', 'Xe_Re_5_T00', 'Xe_Re_R6_T00',
       'Xe_Re_6_T00', 'Xe_Re_7_T00', 'Xe_Re_R7_T00', 'Xe_Re_R2_T01',
       'Xe_Re_R3_T01', 'Xe_Re_R5_T01', 'Xe_Re_6_T01', 'Xe_Re_7_T01',
       'Xe_Re_8_T01', 'Xe_Re_R2_T02

In [102]:
lcms_col_array = np.array(lcms.columns)
lcms_col_array = np.array([replace_third_column(col) for col in lcms_col_array])
uneq=[a != b for a, b in zip(lcms_col_array, lcms.columns)] # check where gcms.columns and gcms_col_arry differ
# compare the orignal columns names with the processed columns
print("unprocessed >>>/n", np.array(lcms.columns[uneq]), "length >>> ", len(lcms.columns))
print("procesed >>>/n", lcms_col_array[uneq], "length >>> ", len(lcms_col_array))
lcms_col_array[0] = "metabolite"
lcms.columns = lcms_col_array
#lcms.to_csv('/home/t44p/PW_rawdata/LC-MS/lc-ms_prep.csv', index=False)


unprocessed >>>/n ['Xe_De_5_T00' 'Xe_De_8_T00' 'Xe_De_9_T00' 'Xe_De_5_T03' 'Xe_De_8_T03'
 'Xe_De_9_T03' 'Xe_De_5_T06' 'Xe_De_8_T06' 'Xe_De_9_T06' 'Xe_De_5_T09'
 'Xe_De_8_T09' 'Xe_De_9_T09' 'Xe_De_5_T12' 'Xe_De_6_T12' 'Xe_De_9_T12'
 'Xe_De_5_T15' 'Xe_De_6_T15' 'Xe_De_8_T15' 'Xe_De_9_T15' 'Xe_De_5_T24'
 'Xe_De_8_T24' 'Xe_De_9_T24' 'Xe_Re_4_T00' 'Xe_Re_5_T00' 'Xe_Re_6_T00'
 'Xe_Re_7_T00' 'Xe_Re_6_T01' 'Xe_Re_7_T01' 'Xe_Re_8_T01' 'Xe_Re_4_T02'
 'Xe_Re_6_T02' 'Xe_Re_7_T02' 'Xe_Re_4_T04' 'Xe_Re_6_T04' 'Xe_Re_7_T04'
 'Xe_Re_4_T08' 'Xe_Re_6_T08' 'Xe_Re_7_T08' 'Xe_Re_4_T12' 'Xe_Re_6_T12'
 'Xe_Re_7_T12' 'Xe_Re_4_T48' 'Xe_Re_6_T48' 'Xe_Re_7_T48'] length >>>  87
procesed >>>/n ['Xe_De_R5_T00' 'Xe_De_R8_T00' 'Xe_De_R9_T00' 'Xe_De_R5_T03'
 'Xe_De_R8_T03' 'Xe_De_R9_T03' 'Xe_De_R5_T06' 'Xe_De_R8_T06'
 'Xe_De_R9_T06' 'Xe_De_R5_T09' 'Xe_De_R8_T09' 'Xe_De_R9_T09'
 'Xe_De_R5_T12' 'Xe_De_R6_T12' 'Xe_De_R9_T12' 'Xe_De_R5_T15'
 'Xe_De_R6_T15' 'Xe_De_R8_T15' 'Xe_De_R9_T15' 'Xe_De_R5_T24'
 'Xe_De_R8_T24' 'Xe_D

# Add sample column to the tr_mut_transposed.csv
done very late. For WGCNA


In [15]:
tr_mut_transposed = pd.read_csv("/home/t44p/PW_rawdata/tr_gc_mutual/tr_mut_transposed.csv", sep=",")
tr_mut = pd.read_csv("/home/t44p/PW_rawdata/tr_gc_mutual/tr_mut.csv", sep=",")

In [16]:
tr_mut.columns[:]

Index(['ref', 'Xe_De_R2_T00', 'Xe_De_R2_T03', 'Xe_De_R2_T06', 'Xe_De_R2_T09',
       'Xe_De_R2_T12', 'Xe_De_R2_T24', 'Xe_De_R3_T00', 'Xe_De_R3_T03',
       'Xe_De_R3_T06', 'Xe_De_R3_T09', 'Xe_De_R3_T12', 'Xe_De_R3_T15',
       'Xe_De_R3_T24', 'Xe_De_R4_T00', 'Xe_De_R4_T03', 'Xe_De_R4_T06',
       'Xe_De_R4_T09', 'Xe_De_R4_T12', 'Xe_De_R4_T15', 'Xe_De_R4_T24',
       'Xe_Re_R2_T00', 'Xe_Re_R2_T01', 'Xe_Re_R2_T02', 'Xe_Re_R2_T04',
       'Xe_Re_R2_T08', 'Xe_Re_R2_T12', 'Xe_Re_R2_T48', 'Xe_Re_R3_T02',
       'Xe_Re_R3_T04', 'Xe_Re_R3_T08', 'Xe_Re_R3_T12', 'Xe_Re_R3_T48',
       'Xe_Re_R5_T01', 'Xe_Re_R5_T02', 'Xe_Re_R5_T04', 'Xe_Re_R5_T08',
       'Xe_Re_R5_T12', 'Xe_Re_R5_T48', 'Xe_Re_R6_T00', 'Xe_Re_R7_T00'],
      dtype='object')

In [17]:
# Extract column names from tr_mut
sample_names = tr_mut.columns[1:].tolist()

# Add the sample names as the first column in tr_mut_transposed
tr_mut_transposed.insert(0, 'ref', sample_names)
tr_mut_transposed.columns[0:4]

Index(['ref', 'Xele.ptg000212l.1', 'Xele.ptg000045l.1', 'Xele.ptg000045l.2'], dtype='object')

In [18]:
# Save the modified DataFrame to a new CSV file
#tr_mut_transposed.to_csv("/home/t44p/PW_rawdata/tr_gc_mutual/tr_mut_transposed_modified.csv", index=False)